In [86]:
dd.to_csv('out.csv')

In [9]:
# coding: utf-8
# pylint: disable = invalid-name, C0111
import json
import lightgbm as lgb
import pandas as pd
import numpy
from sklearn.metrics import mean_squared_error

# load or create your dataset
print('Load data...')
df_train = pd.read_csv('train.csv',sep=';')
df_test = pd.read_csv('test.csv', sep=';')

target = df_train['target'].values

#df_train2 = df_train[['cardreader_error_by_day','status_count']]
columns = ['cash_in_out','display_type','scanner_code_reader']
df_train.drop(columns, inplace=True, axis=1)
train = df_train.values
train = df_train[['cardreader_error_by_day','status_count']]

#df_test2 = df_test[['cardreader_error_by_day','status_count']]
df_test.drop(columns, inplace=True, axis=1)
test = df_test.values
test = df_test[['cardreader_error_by_day','status_count']]

Load data...


In [10]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(train, target)
#lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Start training...')
# train
#gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=5)
gbm = lgb.train(params, lgb_train)

Start training...


In [23]:
print('Save model...')
# save model to file
gbm.save_model('model.txt')

#print('Start predicting...')
# predict
PREDICT = gbm.predict(test, num_iteration=gbm.best_iteration)
# eval
#print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)


Save model...


In [24]:
OUT=pd.DataFrame(numpy.around(PREDICT), columns=['PREDICT']) 

In [25]:
OUT

,PREDICT
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,1.0
6,0.0
7,1.0
8,0.0
9,1.0


In [29]:
OUT.to_csv('OUT.csv', sep=';', encoding='utf-8')

ValueError: Shape of passed values is (351, 1), indices imply (1, 1)